## Content

-  Introduction to Unsupervised Learning and Problem Statement

-  Clustering and its Intuition

-  Dunn Index

-  K-Means and its Math Formulation

-  Lloyd's Algorithm

-  K-Means Scratch Implementation

-  Determining K  and Silhouette Score

-  Amazon's Customer Segmentation with K-Means




Till now, what we learnt or what we did using Machine Learning was supervised.

Meaning, we were given observations $x_{i}$s with their end results/ground truth $y_{i}$s.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/115/original/Screenshot_2022-07-29_at_3.24.57_PM.png?1659088030' height = "550" width = "800">

- For a given dataset $D = {x_i, y_i}$
  - If $y_i∈{0,1}$, then we call it a **2-class classification**, or if $y_i$ consists of discrete values then it is a general classification problem.
  - If $y_i∈R$ or is continuous, then we call it a **regression** problem.

- **In the case of both classification and regression** problems, we are trying to find a function that is used to predict $y_i$, when $x_i's$ are given as the input. Both of these are **supervised learning problems**, where the models are trained with a target variable.

Now, consider a dataset of 100000 people which contains information such as location, gender, number of orders, and account balance.

If someone wants to divide these 100000 people into bunch of groups based on regular, semi-premium and premium customer, how would one that?

**NOTE:** In the dataset, there is no data about somenone being regular/semi-premium/premium customer.

**Q. So, how can one work with the data that is not labelled? Do we need to label the these types of data?**

- Recall from the Machine Learning introductory class, that within Machine Learning, there are two approaches.
  1. Supervised Learning
  2. Unsupervised Learning

The basic and the simplest definition of Unsupervised learning is:
  - Unsupervised learning uses machine learning algorithms to analyze and cluster unlabeled data sets.
  - These algorithms discover hidden patterns in data without the need for human intervention (hence, they are “unsupervised”).

  So, from definition, we can relate a bit from the word 'unlabeled'. Let's see how can we use unsupervised methods to solve a similar problem.

##Problem statement:

As a data scientist at **Amazon**, you are given a dataset that has details about different customers with features like
- 'ID',
- 'n_clicks',
- 'n_visits', etc,

You are asked to segment these customers so that the **Amazon** can provide relevant and similar items to their customers, which will increase their overall sale.

In [ ]:
!wget "https://drive.google.com/uc?export=download&id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W" -O E-commerce.csv

--2023-09-28 08:16:06--  https://drive.google.com/uc?export=download&id=1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W
Resolving drive.google.com (drive.google.com)... 142.251.163.138, 142.251.163.113, 142.251.163.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k9t3lajv689r97nb47o1lj74baf2mslp/1695888900000/10306167880925931714/*/1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W?e=download&uuid=ec048533-7137-485a-8443-d7d3018c88ba [following]
--2023-09-28 08:16:07--  https://doc-10-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k9t3lajv689r97nb47o1lj74baf2mslp/1695888900000/10306167880925931714/*/1lEccW5Y5_2z00VRtLGOAJOAU6YA9fl6W?e=download&uuid=ec048533-7137-485a-8443-d7d3018c88ba
Resolving doc-10-64-docs.googleusercontent.com (doc-10-64-docs.googleusercontent.com)... 172.253.63.132, 2

Let's look at the provided features in our dataset.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('./E-commerce.csv')
df.head()

,ID,n_clicks,n_visits,amount_spent,amount_discount,days_since_registration,profile_information
0,1476,130,65,213.905831,31.600751,233,235
1,1535,543,46,639.223004,5.689175,228,170
2,1807,520,102,1157.402763,844.321606,247,409
3,1727,702,83,1195.903634,850.041757,148,200
4,1324,221,84,180.754616,64.283300,243,259


Let's see what all do we have for the given problem statement:

- First thing to note is that there are no labels for the records in the dataset.

- What this means? We cannot use Supervised Learning methods to solve our problem, and we would have to use Unsupervised methods.


Recall the introductory class of machine learning where we saw that there is a type of unsupervised learning technique called clustering.

**Q. Do you think clustering can be used here?**

- Yes, here we are asked to group customers together who are similar.

##What is clustering?

- At simple as it can be, clustering is a task of grouping similar objects or data-points.

- **For example**:
  - A professional cricket team can collect information like **runs_per_game**, **wickets_per_game**, **catches_per_game**, etc and then use these features to cluster similar players.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/010/002/original/Screenshot_2022-09-08_at_10.49.09_AM.png?1662613855' height = '500' width = '800'>

In the plot you can observe that we have groups or clusters of similar kind of players like batsman, bowlers, allrounders, etc.

## Intuition of Clustering:

- Intuitively, Clustering is the task of dividing the population or data points into a number of groups such that data points which are similar are closer and in the same group than the dissimilar points.

- Each group in clustering is called a **cluster**.
 - The points in the same cluster are more closer and similar to each other.
 - The points in different clusters are more distant and distinct from each other.

- So, the task in clustering is **grouping the points of similar kind** based on **our definition of similarity**.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/120/original/Screenshot_2022-07-29_at_3.43.55_PM.png?1659089170' height = '500' width = '800'>

<br>

**Q. But what is the definition of similarity? How to decide which points are similar?**

- Similarity in clustering can be defined as the closeness of data points with each other

- For example, customers who spend almost equal amount of money on shopping on amazon can be similar.

- Similarity can be measured using different distance metrics.

> **Q. How can we ensure clustering is good or bad?**

- Since **clustering doesn’t have the class labels or the ground truth**, it is hard to measure if a clustering is good or bad in a very rigorous and critical way.

- It all depends on the problem and the context we are working in, i.e., the **Business Case**

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/121/original/Screenshot_2022-07-29_at_3.45.32_PM.png?1659089264' height = '500' width = '800'>

**Clustering, just like classification, is very dependent on the features used.**

- The **clustering output can sometimes help coming up with new features**.

- If we get a **new datapoint** in future, for example, **a new customer at Amazon**, we can identify which cluster or group does this new datapoint or this new customer belongs to.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/122/original/Screenshot_2022-07-29_at_3.46.25_PM.png?1659089315' width="500">

---


**Now that we have seen the intuition of clustering let's look at few terms and concepts we should be aware about during clustering on any dataset.**

###Intra-cluster and Inter-cluster Distance

- Imagine two clusters $C_{1}$ and $C_{2}$ based on some feature $f_{1}$ and $f_{2}$.

- Inter-cluster distance represents the distance between two clusters $C_{1}$ and $C_{2}$.

- There are many ways to calculate the distance between two clusters. Such as:
  - Distance between average values of the clusters.
  - Distance between closest points from the clusters (min distance)
  - Distance between farthest points from the clusters (max distance)

- Intra-cluster distance represents the distance within a certain cluster. Basically , it measures how tightly the points of a clusters are packed.

- Again there are more than one way to measure intra-cluster distance:
  - Average distance between the points of a cluster.
  - Distance between farthest points of a cluster

**Q. How to determine which approach to use to calculate these distances?**
- There is no right or wrong approach. All these ideas are valid and which one to choose depends on the use-case and domain that we're working on.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/124/original/Screenshot_2022-07-29_at_3.49.42_PM.png?1659089519' height = 500 width = 800>

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/010/017/original/Screenshot_2022-09-08_at_10.03.24_PM.png?1662654263' width="500">


**Q. What should be the ideal values of inter and intra cluster distances of a clustering?**

- Ideally, **intra-cluster** distance should be **low**
- If the intra cluster distance will be low, the points in the same clusters will be more similar

- And, **inter-cluster** distance should be **high**.
- Incase of higher inter cluster distances, the points in different clusters will be less similar.

#### Q. How can we calculate the distances between the points and clusters?

Recall from the pre-read, we can use the following distances for calculating the similarity:-
- **Euclidean distance**
 - It is preferred in low dimensional space.
- **Manhattan distance**
 - It is used in low-medium dimensional space
- **Cosine similarity**
 - It is used in high-dimensional space.

Practically speaking distance-metric used is hyperparameter and is **application based.**


<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/299/original/Screenshot_2022-08-02_at_3.34.46_PM.png?1659434213' height = '500' width = '800'>

**Q. If only one of the inter-cluster distance or intra-cluster distance is given, can you really judge how good or bad the formed clusters are?**

Only having one of inter or intra cluster distance does not work because:

- It is not necessay that if inter cluster distance is very high, intra-cluster will be low

- Similarly just having a smaller intra cluster distance also does not guarantee good clustering results.


## Q. Then what metric can we use to evaluate our Clustering?

- Ideally, the resultant clusters should make **business sense**

- Say that you are working for a shirt manufacturing company, then you should have 3 clusters: small, medium, large.

- It does not make sense to have 100 clusters!

- To evaluate the results of clustering, we can also use the same distance metrics we used for supervised learning problems, like Euclidean Distance,etc.


<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/123/original/Screenshot_2022-07-29_at_3.47.54_PM.png?1659089419' height = 500 width = 800>

Let's look at one more metric which includes inter and intra cluster distance to evaluate clustering.

### **Dunn Index**

- It is a metric for evaluating clustering algorithms
- The objective of Dunn index is to identify clusters that are:
 - compact with a small variance between members of the cluster
 - and well separated.

**Q. How can we calculate Dunn Index?**
- It is denoted by **‘D’** and is given as:

$D = \frac{min_{i,j} distance(i,j)}{max_k distance^{'}(k)}$

where;
- $distance(i,j)$ → distance between the farthest points of the clusters $C_i$ and $C_j$ → **Inter-Cluster distance**

- ${distance^{'}(k)}$ → distance between the farthest points within the $k^{th}$ clusters **Intra-Cluster distance**

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/125/original/Screenshot_2022-07-29_at_3.53.47_PM.png?1659089763' height = '500' width = '800'>


<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/126/original/Screenshot_2022-07-29_at_3.56.07_PM.png?1659089899' height = '500' width = '800'>



> **Q. How can we interpret the values of Dunn index?**

- If $Dunn-index$ is high, it implies that clusters are well separated and the points in the same cluster are intact.
- For every pair of points from $C_i$ and $C_j$, we have to compute **$distance(i,j)$** for getting the inter cluster distance.
- Similarly for calculating the **$distance'(k)$** we will have to iterate through each pair of points within $k^{th}$ cluster

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/127/original/Screenshot_2022-07-29_at_3.57.44_PM.png?1659089997' height = '500' width = '800'>


###Quiz-2:
```
We have two clustering algorithms, and if we have to choose one of them, the algorithms chosen should have:
a. higher dunn-index
b. lower dunn-index
```
Answer. a

- **For ideal clustering, the value of the Dunn Index should be high**

- For this, the distances between the points in the same cluster should be lower, and the distance between the different clusters should be higher.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/128/original/Screenshot_2022-07-29_at_4.01.33_PM.png?1659090224' height = '500' width = '800'>

- If we have two Clustering algorithms and we need to decided which one is performing better, we can compare the Dunn Index of the two algorithms and pick the one with higher Dunn Index.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/129/original/Screenshot_2022-07-29_at_4.02.25_PM.png?1659090275' height = '500' width = '800'>

**But why we chose maximum intra-cluster distance and minimum inter-cluster distance to use in Dunn-index?**

- max intra-cluster and min inter-cluster distances are chosen in order to handle the extreme worst case scenarios.

Consider the following two cases, which do you think represent better clustering? Case1 or Case2.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/227/original/Screenshot_2022-08-01_at_1.58.35_PM.png?1659342098' height = '500' width = '800'>

- If we'd have chosen max inter-cluster distance,
 - metric of case1 will be higher but it shouldn't be the case as it's not the ideal situation,
 - here the 4th cluster in case1 is just an outlier and its extremely affecting the metric due to which we use minimum of inter-cluster distances.

 - incase of max inter-cluster, it would always choose the clustering algorithm where one cluster is far from the other clusters and the remaining clusters are very close to each other, which is not correct.


**Is there any range of Dunn-index, how to say if a value of Dunn-index is good or bad?**
- There is no such good or bad range of values for Dunn-index.
- For an instance say your min inter-cluster distance is 10 times the max intra-cluster distance, it will be a good clustering but its just a comparative analysis.
- For analysing models using Dunn-index:
 - we can find the values of the metric for a random clustering algorithm and      
 - then compare it with the values of metric for different clustering algorithms to find out whether the algorithm is performing better or worse than a random model.
---

**An example to show the use-case of clustering**

##Smart labeling

**Say you are given 10 million images of wearing apparel and you have to label each of them.**

For each image you are given a d-dimension vector using some deep learning algorithm.

> **Q. Will it be feasible to do it manually?**

No, but we can use clustering here.

**Step-1**. We can firstly find clusters from the whole dataset. Say here we find some 100-odd clusters.

**Step-2**. Then we can manually label the centroid points first. Say we label a centroid as 'men's jeans'.

**Step-3**. Next we can label the points which are significantly closer to the centroid of the cluster with the same label as that of centroid. i.e. The points which are closer to the centroid labelled 'men's jeans' will also get labelled as 'men's jeans'.

**Step-4**. This way we can get some part of the data labelled if not all of it.


After this classification techniques can be used to label the remaining data. This won't give perfect labels but can provide reasonably good labels.

This is one of the best applications of clustering. Google also uses this while speech to text conversion where it takes snippets of audio samples and look at the similar snippets and label only few of available snippets.

This will get discussed in depth further in the course.

Now that we've understood basic intuition of clustering, let's dive a little deep and understand an algorithm of clustering- K-Means.

---

## K-Means Clustering introduction

- K-Means clustering is one of the most popular and the simplest clustering
algorithms. The value **'K' in the K-means algorithm denotes the number of
clusters**.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/130/original/Screenshot_2022-07-29_at_4.04.01_PM.png?1659090377' height = '500' width = '800'>

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/131/original/Screenshot_2022-07-29_at_4.05.12_PM.png?1659090450' height = '500' width = '800'>



- Consider a 2-Dimensional dataset $D$ with number
of clusters (K) = 3. So the number of centroids should also be equal to 3.

- A centroid of a cluster is the middle point of the cluster which can be calculated using average of points.

- Let 'S1', 'S2', 'S3' be different sets of elements of different clusters, and ‘C1’, ‘C2’ and ‘C3’ be their respective cluster centroids.

  $S_1 ∪ S_2 ∪ S_3 = D$

  $S_1 ∩ S_2 ∩ S_3 = Φ$

  $S_1 ∩ S_2 = Φ, S_1 ∩ S_3 = Φ, S_2 ∩ S_3 = Φ$



- **Q. What does this indicate?**

  - This indicates the fact that all the data points belong to one or the other set and no data point exists in more than one set or cluster.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/132/original/Screenshot_2022-07-29_at_4.07.21_PM.png?1659090573' height = '500' width = '800'>

Thus,

Number of Sets = K (i.e., $S_1, S_2, S_3, ….., S_k$), and

Number of centroids = K (i.e., $C_1, C_2, C_3, ….., C_k$).


Now, consider a set of points $S_i$ of size $|S_i|$. We can find the centroid of this set by summing all the points in the set (x_j) and averaging them by the size of the set ($|S_i|$).

  - $C_i$ = $\Large\frac{1}{|S_i|} ∑_{x_j ∈ S_i} x_j$

Now, K-means clustering is a **centroid-based** clustering scheme.

- Every point is assigned to the cluster closest to it.

- The core idea of K-means clustering is to find 'K' centroids and each point is assigned to the cluster whose centroid is nearest to it.

- The biggest challenge is to find these 'K' centroids.

- There are algorithms to find out these 'K' centroids and one of the most commonly used algorithms is **Lloyd's algorithm**.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/133/original/Screenshot_2022-07-29_at_4.08.31_PM.png?1659090646' height = '500' width = '800'>

Now that we've seen  a high level approach, let's now move to mathematical formulation of K-Means.

## K-Means: Mathematical Formulation: Objective Function

**Q. What do we optimize in K-Means? What is our end-goal?**

- Given a dataset $D$ where each point $x_i ∈ R^d$, we want to find clusters '$S_1$', '$S_2$’, '$S_3$', …., '$S_k$' and their corresponding centroids '$C_1$', '$C_2$', '$C_3$', …., '$C_k$' such that;
  1. the inter-cluster distance is maximum
  2. the intra-cluster distance is minimum


<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/134/original/Screenshot_2022-07-29_at_4.10.14_PM.png?1659090747' height = '500' width = '800'>


**Since we're trying to optimization problem, Can we use Gradient Descent to converge the K-Means Clustering Algorithm?**

- In clustering, clusters are always represented using discrete variables.

- When we apply Gradient Descent the variables can become fraction which shouldn't be the case as it should only be discrete values.

- The values cannot be a fraction as a point can either belong to a set or not, so GD does not work. And, which is why Gradient Descent fails here.



**Q. What is the problem with value of clustering variables being fractional?**

- If variable is fractional,
 - it would mean that the a particular point is partially belonging to ith cluster as well as some other cluster as well

 - This kind of clustering is called **Fuzzy-clustering**/**Soft-clustering**.

- To solve this optimization problem with the integer constraints we will have to use the integer programming which will have NP-hard or exponential complexity.

- This optimization problem is very hard to solve due to which it's not used in real life applications.

- In such cases where the complexity of problem becomes very large, we optimize with the approximation algorithms and find out nearest solutions to the problems.

One such approximation algorithm is **Lloyd’s algorithm**. It is a very simple and a good approximation algorithm.

- Approximation algorithms do not guarantee best solutions but they work well in practice.

So, let's see how this works. Coming up is Llyod's Algorithm.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/140/original/Screenshot_2022-07-29_at_4.15.24_PM.png?1659091061' height = '500' width = '800'>

---



## K-Means Algorithm (Lloyd’s Algorithm)

Lloyd' algorithm is an approximation method we use for k-means clustering, which involve 4 major steps:


**1. Initialization**

From the given dataset '**D**', we pick '**K**' points randomly, and assume them to be the initial centroids. Let us denote them as $C_1, C_2, C_3, …, C_k$.

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/141/original/Screenshot_2022-07-29_at_4.18.21_PM.png?1659091230' height = '500' width = '800'>


**2. Assignment**

For each point '$x_i$' in the dataset '**D**', we have to compute the distance of each of the above 'K' centroids, and pick the nearest centroid. Let us denote this nearest centroid as '$C_j$'.

Add the point '$x_i$' to the set '$S_j$'(which is associated with the centroid '$C_j$').

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/142/original/Screenshot_2022-07-29_at_4.19.16_PM.png?1659091285' height = '500' width = '800'>


**3. Recompute Centroid (Update Stage)**

- Now  that we've grouped all the datapoints to each clusters, we update the centroid for each and every clusters.


- We recompute/update '$C_j$' as follows:

    - $C_j = (1/|S_j|) * Σ_{x_i∈S_j} x_i$

<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/143/original/Screenshot_2022-07-29_at_4.20.53_PM.png?1659091397' height = '500' width = '800'>


**4. Repeat the assignment and update steps until convergence**.

**Q. How do we know if algorithm has converged?**

- Here convergence is the stage where the centroids do not change much.

- For example, after second iteration, if the centroids are $\{C_1, C_2, C_3, …., C_k\}$ and 3rd iteration, if the updated centroids are $C_1$', $C_2$', $C_3$', ….., $C_k$', we say the algorithm has converged if the distance between old centroids and updated centroids is very low.


<img src='https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/008/144/original/Screenshot_2022-07-29_at_4.22.29_PM.png?1659091481' height = '500' width = '800'>


- After convergence, we get centroids as $C_1, C_2, C_3, …., C_k$ and the final sets/clusters as $S_1, S_2, S_3, …., S_k$.


Now, let's see how to implement K-Means from scratch.